# nb-model_exp-results-5

In [1]:
import sys
import os
from glob import glob
from os import sep
from os.path import dirname, realpath, exists, basename
from pathlib import Path
from glob import glob, iglob
from distutils.dir_util import copy_tree
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_exp-results-5.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
import optuna

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 50)

from common_util import MODEL_DIR, dump_df, benchmark, NestedDefaultDict, load_json, makedir_if_not_exists
from model.common import ASSETS, INTERVAL_YEARS, INTRADAY_LEN, EXP_LOG_DIR, EXP_PARAMS_DIR
from model.common import OPTUNA_DB_FNAME, OPTUNA_N_TRIALS, OPTUNA_TIMEOUT_HOURS
#from common_util import MODEL_DIR, load_json, dump_json, rectify_json, load_df, str_now, makedir_if_not_exists, is_valid, isnt, compose, pd_split_ternary_to_binary, df_del_midx_level, midx_intersect, pd_get_midx_level, pd_rows, df_midx_restack
#from common_util import NestedDefaultDict
#from model.common import EXP_LOG_DIR, EXP_PARAMS_DIR, ASSETS, DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, TRAIN_RATIO, EXPECTED_NUM_HOURS
from recon.viz import *
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_exp-results-5.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Optuna (exp_orun) Study Results

In [2]:
sm_name = 'anp'
model_name = 'np'
test_final_dir = EXP_PARAMS_DIR +'final' +sep
opt_dir = EXP_LOG_DIR+'optuna'+sep
sm_dir = opt_dir+sm_name+sep
params_dict = load_json('exp_orun.json', MODEL_DIR)

In [3]:
def get_ostudy_name(study_dir, base_dir=EXP_LOG_DIR+'optuna'+sep):
	return study_dir.replace(base_dir, '').replace(sep, ',')

In [4]:
def df_top(study_df, n=10):
	top = study_df.nlargest(n, 'value', keep='all')
	return top

def df_completed(study_df):
	return study_df.loc[study_df['state'] == 'COMPLETE']

### Load all Studies

In [5]:
ndd_study = NestedDefaultDict()

# glob all studies:
for asset_name, v1 in params_dict.items():
	for data_name, param_names in v1.items():
		for param_name in param_names:
			for path in glob(f"{sm_dir}/**/{asset_name}/{model_name}/{data_name}/{param_name}/**/trials.db", recursive=True):
				study_path = path.replace('trials.db', '')
				ndd_study[[asset_name, data_name, param_name]] = get_ostudy_name(study_path), study_path

### Display All Study Top Results

In [6]:
for path in ndd_study.keys():
	study_name, study_dir = ndd_study[path]
	study_db_path = f'sqlite:///{study_dir}{OPTUNA_DB_FNAME}'
	study_df = optuna.load_study(storage=study_db_path, study_name=study_name) \
		.trials_dataframe().sort_values(by='value', ascending=False)
	print(path)
	display(df_top(study_df))

['dow_jones', '2007_2018_ddir_h_pba_h', 342]


,number,value,datetime_start,datetime_complete,duration,params_de_depth,params_de_global_dropout,params_de_init,params_de_size_pwr,params_det_class_agg,params_det_rt_num_heads_pwr,params_det_xa_num_heads_pwr,params_ft_global_dropout,params_ft_init,params_ft_size,params_lat_class_agg,params_lat_rt_num_heads_pwr,params_sample_latent_post,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,system_attrs_completed_rung_3,system_attrs_fixed_params,state
644,644,1.742176,2021-10-15 12:14:34.976874,2021-10-15 12:15:57.747179,0 days 00:01:22.770305,3.0,0.3,xavier_uniform,0.0,True,2.0,0.0,0.1,kaiming_uniform,4.0,False,1.0,False,1.742176,1.742176,1.742176,1.742176,NaN,COMPLETE
675,675,1.712354,2021-10-15 12:25:54.779543,2021-10-15 12:27:37.937694,0 days 00:01:43.158151,2.0,0.5,xavier_normal,2.0,True,1.0,0.0,0.3,kaiming_uniform,12.0,True,2.0,False,1.712354,NaN,NaN,NaN,NaN,COMPLETE
163,163,1.693881,2021-10-15 08:28:14.604379,2021-10-15 08:29:38.719064,0 days 00:01:24.114685,2.0,0.5,kaiming_uniform,1.0,True,1.0,0.0,0.5,kaiming_uniform,8.0,False,2.0,True,1.693881,1.693881,1.693881,NaN,NaN,COMPLETE
923,923,1.624897,2021-10-15 14:17:26.466429,2021-10-15 14:18:49.861314,0 days 00:01:23.394885,3.0,0.5,xavier_uniform,1.0,True,1.0,0.0,0.1,xavier_uniform,4.0,False,1.0,False,1.624897,1.624897,1.624897,1.624897,NaN,COMPLETE
449,449,1.603781,2021-10-15 10:53:33.935351,2021-10-15 10:55:02.987234,0 days 00:01:29.051883,3.0,0.3,xavier_uniform,1.0,True,2.0,0.0,0.1,kaiming_uniform,4.0,False,1.0,False,1.603781,1.603781,1.603781,1.603781,NaN,COMPLETE
515,515,1.566759,2021-10-15 11:26:20.308238,2021-10-15 11:27:53.342296,0 days 00:01:33.034058,2.0,0.3,xavier_normal,2.0,True,1.0,0.0,0.3,xavier_uniform,8.0,True,0.0,False,1.566759,NaN,NaN,NaN,NaN,COMPLETE
561,561,1.534629,2021-10-15 11:44:25.470685,2021-10-15 11:45:41.130060,0 days 00:01:15.659375,3.0,0.3,xavier_normal,2.0,True,0.0,2.0,0.5,kaiming_uniform,4.0,False,2.0,False,1.534629,1.534629,1.534629,NaN,NaN,COMPLETE
571,571,1.504681,2021-10-15 11:48:28.477702,2021-10-15 11:49:48.085245,0 days 00:01:19.607543,3.0,0.3,xavier_uniform,0.0,True,2.0,0.0,0.3,kaiming_uniform,4.0,False,1.0,False,1.504681,1.504681,1.504681,1.504681,NaN,COMPLETE
945,945,1.504380,2021-10-15 14:26:39.377303,2021-10-15 14:28:10.460108,0 days 00:01:31.082805,3.0,0.3,xavier_uniform,0.0,True,1.0,0.0,0.1,kaiming_uniform,4.0,False,1.0,False,1.504380,1.504380,1.504380,1.504380,NaN,COMPLETE
829,829,1.500033,2021-10-15 13:30:36.682951,2021-10-15 13:31:57.844171,0 days 00:01:21.161220,3.0,0.5,xavier_uniform,1.0,True,1.0,0.0,0.1,kaiming_uniform,4.0,False,1.0,False,1.500033,1.500033,1.500033,1.500033,NaN,COMPLETE


['dow_jones', '2007_2018_ddir_h_pba_h_h_vol_h', 445]


,number,value,datetime_start,datetime_complete,duration,params_de_depth,params_de_global_dropout,params_de_init,params_de_size_pwr,params_det_class_agg,params_det_rt_num_heads_pwr,params_det_xa_num_heads_pwr,params_ft_global_dropout,params_ft_init,params_ft_size,params_lat_class_agg,params_lat_rt_num_heads_pwr,params_sample_latent_post,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,system_attrs_completed_rung_3,system_attrs_fixed_params,state
339,339,2.440266,2021-10-17 08:35:15.923892,2021-10-17 08:36:48.402710,0 days 00:01:32.478818,2.0,0.1,xavier_uniform,2.0,False,0.0,1.0,0.5,xavier_normal,12.0,True,2.0,False,1.921275,NaN,NaN,NaN,NaN,COMPLETE
463,463,2.339264,2021-10-17 09:39:06.508841,2021-10-17 09:40:38.627758,0 days 00:01:32.118917,2.0,0.1,xavier_uniform,2.0,False,0.0,1.0,0.5,xavier_normal,12.0,True,2.0,False,2.115201,NaN,NaN,NaN,NaN,COMPLETE
153,153,2.130579,2021-10-17 07:05:25.238679,2021-10-17 07:06:58.951666,0 days 00:01:33.712987,3.0,0.1,kaiming_uniform,2.0,False,0.0,1.0,0.5,xavier_normal,12.0,True,2.0,False,1.255765,NaN,NaN,NaN,NaN,COMPLETE
944,944,2.100050,2021-10-17 13:56:13.780871,2021-10-17 13:57:55.650199,0 days 00:01:41.869328,2.0,0.1,xavier_uniform,2.0,False,0.0,0.0,0.3,xavier_normal,12.0,True,2.0,True,1.615178,NaN,NaN,NaN,NaN,COMPLETE
461,461,2.080135,2021-10-17 09:37:29.136740,2021-10-17 09:39:01.580812,0 days 00:01:32.444072,2.0,0.1,xavier_uniform,2.0,False,0.0,1.0,0.5,xavier_normal,12.0,True,2.0,False,1.856859,NaN,NaN,NaN,NaN,COMPLETE
99,99,2.079085,2021-10-17 06:45:12.209226,2021-10-17 06:46:49.624340,0 days 00:01:37.415114,2.0,0.1,xavier_uniform,2.0,False,0.0,0.0,0.5,kaiming_uniform,12.0,True,2.0,False,1.433923,NaN,NaN,NaN,NaN,COMPLETE
172,172,2.035702,2021-10-17 07:15:59.289612,2021-10-17 07:17:31.985942,0 days 00:01:32.696330,3.0,0.1,kaiming_uniform,2.0,False,0.0,1.0,0.5,xavier_normal,12.0,True,2.0,False,1.573930,NaN,NaN,NaN,NaN,COMPLETE
173,173,2.029711,2021-10-17 07:17:32.025065,2021-10-17 07:19:03.871949,0 days 00:01:31.846884,2.0,0.1,xavier_uniform,2.0,False,0.0,1.0,0.5,xavier_normal,12.0,True,2.0,False,1.460668,NaN,NaN,NaN,NaN,COMPLETE
737,737,1.997123,2021-10-17 12:04:35.575818,2021-10-17 12:06:16.796845,0 days 00:01:41.221027,3.0,0.1,xavier_uniform,1.0,False,0.0,0.0,0.5,kaiming_uniform,12.0,True,2.0,False,2.055158,NaN,NaN,NaN,NaN,COMPLETE
643,643,1.983687,2021-10-17 11:25:34.578524,2021-10-17 11:27:23.877592,0 days 00:01:49.299068,3.0,0.5,xavier_uniform,2.0,True,2.0,1.0,0.3,xavier_normal,12.0,True,0.0,True,1.983687,1.983687,1.983687,1.983687,NaN,COMPLETE


['dow_jones', '2007_2018_ddir_h_pba_h_h_vol_h', 447]


...

,number,value,datetime_start,datetime_complete,duration,params_de_depth,params_de_global_dropout,params_de_init,params_de_size_pwr,params_det_class_agg,params_det_rt_num_heads_pwr,params_det_xa_num_heads_pwr,params_ft_global_dropout,params_ft_init,params_ft_size,params_lat_class_agg,params_lat_rt_num_heads_pwr,params_sample_latent_post,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,system_attrs_completed_rung_3,system_attrs_fixed_params,state
473,473,2.305714,2021-10-13 23:43:45.464804,2021-10-13 23:44:53.097970,0 days 00:01:07.633166,3.0,0.3,kaiming_uniform,2.0,True,2.0,1.0,0.5,kaiming_uniform,4.0,True,1.0,True,2.186791,2.219773,2.334952,2.165664,NaN,COMPLETE
430,430,1.934645,2021-10-13 23:29:19.222589,2021-10-13 23:30:27.307599,0 days 00:01:08.085010,3.0,0.5,kaiming_uniform,2.0,True,2.0,1.0,0.5,kaiming_uniform,4.0,True,0.0,True,1.965147,1.756917,1.710799,1.911668,NaN,COMPLETE
979,979,1.898846,2021-10-14 02:11:16.008360,2021-10-14 02:12:19.895496,0 days 00:01:03.887136,2.0,0.5,xavier_uniform,1.0,True,2.0,1.0,0.5,kaiming_uniform,4.0,False,1.0,False,1.953948,1.934095,1.934095,1.934095,NaN,COMPLETE
585,585,1.864475,2021-10-14 00:13:48.632193,2021-10-14 00:14:58.338529,0 days 00:01:09.706336,3.0,0.3,xavier_uniform,2.0,True,2.0,2.0,0.5,kaiming_uniform,8.0,True,1.0,False,1.556075,1.615897,1.682019,1.708481,NaN,COMPLETE
507,507,1.807097,2021-10-13 23:51:04.473179,2021-10-13 23:52:09.881298,0 days 00:01:05.408119,3.0,0.5,kaiming_uniform,2.0,True,0.0,2.0,0.3,xavier_uniform,4.0,False,1.0,True,1.807097,1.758445,NaN,NaN,NaN,COMPLETE
111,111,1.751450,2021-10-13 21:56:48.928544,2021-10-13 21:57:51.693984,0 days 00:01:02.765440,3.0,0.5,kaiming_uniform,1.0,False,0.0,2.0,0.3,xavier_uniform,4.0,False,0.0,True,0.829809,1.143478,NaN,NaN,NaN,COMPLETE
356,356,1.724161,2021-10-13 23:11:02.444125,2021-10-13 23:12:13.444092,0 days 00:01:10.999967,3.0,0.3,kaiming_uniform,2.0,True,2.0,2.0,0.5,xavier_normal,12.0,True,1.0,True,1.744456,1.749185,1.714783,1.812748,NaN,COMPLETE
106,106,1.707903,2021-10-13 21:54:57.670160,2021-10-13 21:56:05.335896,0 days 00:01:07.665736,3.0,0.5,kaiming_uniform,1.0,True,2.0,1.0,0.5,xavier_normal,4.0,True,0.0,False,1.828205,1.829231,1.828205,1.829231,NaN,COMPLETE
655,655,1.670098,2021-10-14 00:35:27.765576,2021-10-14 00:36:34.132272,0 days 00:01:06.366696,3.0,0.5,kaiming_uniform,2.0,True,0.0,2.0,0.5,xavier_uniform,8.0,False,0.0,False,1.826284,2.026169,NaN,NaN,NaN,COMPLETE
330,330,1.619110,2021-10-13 22:54:57.642800,2021-10-13 22:56:03.880772,0 days 00:01:06.237972,2.0,0.1,xavier_uniform,0.0,True,1.0,2.0,0.5,xavier_uniform,4.0,True,2.0,False,1.542205,1.747410,1.679981,NaN,NaN,COMPLETE


['sp_500', '2007_2018_ddir_h_pba_h_h_vol_h', 376]


,number,value,datetime_start,datetime_complete,duration,params_de_depth,params_de_global_dropout,params_de_init,params_de_size_pwr,params_det_class_agg,params_det_rt_num_heads_pwr,params_det_xa_num_heads_pwr,params_ft_global_dropout,params_ft_init,params_ft_size,params_lat_class_agg,params_lat_rt_num_heads_pwr,params_sample_latent_post,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,system_attrs_completed_rung_3,system_attrs_fixed_params,state
874,874,1.958129,2021-10-15 19:55:08.955947,2021-10-15 19:56:31.670892,0 days 00:01:22.714945,2.0,0.5,xavier_uniform,2.0,True,2.0,2.0,0.3,xavier_normal,4.0,True,1.0,False,1.958129,1.930587,2.006091,2.108571,NaN,COMPLETE
941,941,1.893041,2021-10-15 20:17:19.698600,2021-10-15 20:18:39.306939,0 days 00:01:19.608339,3.0,0.5,kaiming_uniform,0.0,True,1.0,1.0,0.3,kaiming_uniform,4.0,False,1.0,True,1.893041,1.893041,1.893041,NaN,NaN,COMPLETE
966,966,1.871793,2021-10-15 20:23:45.464649,2021-10-15 20:25:04.193642,0 days 00:01:18.728993,2.0,0.5,xavier_normal,0.0,True,1.0,1.0,0.3,kaiming_uniform,4.0,False,1.0,True,1.801345,1.993554,1.895980,NaN,NaN,COMPLETE
161,161,1.823976,2021-10-15 15:58:12.862580,2021-10-15 15:59:51.026857,0 days 00:01:38.164277,2.0,0.5,xavier_normal,0.0,True,1.0,1.0,0.1,xavier_uniform,4.0,True,1.0,False,1.823976,1.696688,1.935891,NaN,NaN,COMPLETE
552,552,1.786600,2021-10-15 18:09:24.912950,2021-10-15 18:10:49.209994,0 days 00:01:24.297044,3.0,0.5,xavier_uniform,2.0,True,2.0,2.0,0.5,xavier_normal,8.0,True,1.0,True,1.912820,1.883342,1.918559,1.855015,NaN,COMPLETE
371,371,1.784710,2021-10-15 17:12:01.204488,2021-10-15 17:13:28.481586,0 days 00:01:27.277098,3.0,0.3,xavier_uniform,2.0,True,2.0,2.0,0.5,xavier_normal,8.0,True,2.0,False,1.730017,1.746510,1.746510,1.783990,NaN,COMPLETE
228,228,1.771792,2021-10-15 16:18:15.767329,2021-10-15 16:19:56.484715,0 days 00:01:40.717386,3.0,0.5,xavier_uniform,0.0,True,1.0,0.0,0.1,kaiming_uniform,8.0,True,1.0,False,1.771792,1.771792,1.771792,NaN,NaN,COMPLETE
698,698,1.759926,2021-10-15 18:58:05.294206,2021-10-15 18:59:26.812514,0 days 00:01:21.518308,2.0,0.1,xavier_uniform,2.0,True,2.0,1.0,0.5,xavier_uniform,8.0,False,2.0,True,1.759926,1.764699,1.759926,1.759926,NaN,COMPLETE
901,901,1.752862,2021-10-15 20:02:35.699403,2021-10-15 20:04:04.163728,0 days 00:01:28.464325,2.0,0.5,xavier_uniform,1.0,True,2.0,0.0,0.5,xavier_normal,8.0,True,1.0,True,1.838673,1.632550,1.943382,1.626384,NaN,COMPLETE
190,190,1.747158,2021-10-15 16:08:37.936376,2021-10-15 16:10:11.036574,0 days 00:01:33.100198,2.0,0.3,xavier_uniform,2.0,True,2.0,2.0,0.5,xavier_uniform,8.0,True,2.0,True,1.566787,1.742989,1.684417,1.774893,NaN,COMPLETE


['sp_500', '2007_2018_ddir_h_pba_h_h_vol_h', 482]


,number,value,datetime_start,datetime_complete,duration,params_de_depth,params_de_global_dropout,params_de_init,params_de_size_pwr,params_det_class_agg,params_det_rt_num_heads_pwr,params_det_xa_num_heads_pwr,params_ft_global_dropout,params_ft_init,params_ft_size,params_lat_class_agg,params_lat_rt_num_heads_pwr,params_sample_latent_post,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,system_attrs_completed_rung_3,system_attrs_fixed_params,state
688,688,1.977277,2021-11-15 18:45:28.606048,2021-11-15 18:47:04.882739,0 days 00:01:36.276691,2.0,0.3,kaiming_uniform,1.0,False,0.0,2.0,0.1,xavier_uniform,8.0,True,2.0,False,1.850646,NaN,NaN,NaN,NaN,COMPLETE
252,252,1.857597,2021-11-15 14:21:47.227411,2021-11-15 14:23:29.178220,0 days 00:01:41.950809,2.0,0.1,kaiming_uniform,1.0,True,1.0,0.0,0.1,xavier_normal,8.0,False,2.0,False,1.857597,1.857597,NaN,NaN,NaN,COMPLETE
202,202,1.754603,2021-11-15 13:58:42.608563,2021-11-15 14:00:11.512885,0 days 00:01:28.904322,2.0,0.1,kaiming_uniform,1.0,True,1.0,2.0,0.3,kaiming_uniform,12.0,False,1.0,True,1.754603,1.754603,1.754603,1.754603,NaN,COMPLETE
347,347,1.735163,2021-11-15 15:18:49.907358,2021-11-15 15:20:16.123560,0 days 00:01:26.216202,2.0,0.3,xavier_uniform,0.0,True,2.0,2.0,0.3,xavier_normal,12.0,False,2.0,True,1.735163,1.735163,1.735163,1.735163,NaN,COMPLETE
120,120,1.718607,2021-11-15 13:22:25.379495,2021-11-15 13:23:50.815075,0 days 00:01:25.435580,3.0,0.5,xavier_uniform,0.0,True,2.0,2.0,0.3,xavier_normal,12.0,False,2.0,False,1.718607,1.718607,1.718607,1.718607,NaN,COMPLETE
403,403,1.701147,2021-11-15 15:44:27.104097,2021-11-15 15:45:58.173566,0 days 00:01:31.069469,3.0,0.1,kaiming_uniform,0.0,True,0.0,2.0,0.3,xavier_uniform,12.0,False,2.0,True,1.701147,1.701147,1.701147,1.701147,NaN,COMPLETE
411,411,1.547956,2021-11-15 15:53:05.973229,2021-11-15 15:54:36.463988,0 days 00:01:30.490759,3.0,0.1,kaiming_uniform,1.0,True,0.0,2.0,0.3,xavier_normal,12.0,False,2.0,True,1.547956,1.547956,1.547956,1.547956,NaN,COMPLETE
847,847,1.534479,2021-11-15 20:24:51.641567,2021-11-15 20:26:40.248906,0 days 00:01:48.607339,3.0,0.1,kaiming_uniform,1.0,False,0.0,2.0,0.1,xavier_uniform,8.0,True,2.0,False,1.163966,NaN,NaN,NaN,NaN,COMPLETE
728,728,1.534226,2021-11-15 19:13:08.229453,2021-11-15 19:14:39.155748,0 days 00:01:30.926295,2.0,0.3,kaiming_uniform,1.0,False,0.0,2.0,0.3,xavier_uniform,8.0,True,2.0,False,1.278923,NaN,NaN,NaN,NaN,COMPLETE
546,546,1.505972,2021-11-15 17:29:47.766669,2021-11-15 17:31:25.549144,0 days 00:01:37.782475,2.0,0.1,xavier_uniform,0.0,True,2.0,1.0,0.5,xavier_normal,12.0,False,2.0,True,1.505972,1.505972,1.505972,1.505972,NaN,COMPLETE


### Dump Top N

In [9]:
n = 30
for path in ndd_study.keys():
    study_name, study_dir = ndd_study[path]
    study_db_path = f'sqlite:///{study_dir}{OPTUNA_DB_FNAME}'
    best_trials = optuna.load_study(storage=study_db_path, study_name=study_name) \
        .trials_dataframe().sort_values(by='value', ascending=False).index[:n]
    print(path)
    for i, trial_num in enumerate(best_trials):
        trial_dir = f'{study_dir}{str(trial_num).zfill(6)}{sep}'
        test_dir = test_final_dir +study_dir.replace(opt_dir, '') +f'{str(i)}_{str(trial_num).zfill(6)}{sep}'
        makedir_if_not_exists(test_dir +"chk")
        copy_tree(trial_dir, test_dir)
        print(f'dumped {test_dir}')

['sp_500', '2007_2018_ddir_h_pba_h_h_vol_h', 482]
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_h_h_vol_h/482/val_binary_sharpe/0_000688/
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_h_h_vol_h/482/val_binary_sharpe/1_000252/
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_h_h_vol_h/482/val_binary_sharpe/2_000202/
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_h_h_vol_h/482/val_binary_sharpe/3_000347/
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_h_h_vol_h/482/val_binary_sharpe/4_000120/
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_h_h_vol_h/482/val_binary_sharpe/5_000403/
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_h_h_vol_h/482/val_binary_sharpe/6_000411/
dumped /home/kev/crunch/model/exp-params/final/anp/sp_500/np/2007_2018_ddir_h_pba_